## 简介

迁移学习主要是将预训练模型（通常在ImageNet等大型高难度数据集上训练过）的一部分作为起点，通过一些改动获得对新项目有用的信息。迁移学习主要分为两种：1）特征提取（feature extraction）和2）微调（fine-tuning）。

### 特征提取（Feature Extraction）

<div>
<img src="https://pyimagesearch.com/wp-content/uploads/2019/05/transfer_learning_keras_feature_extract.png" width="400"/>
</div>

特征提取主要是将预训练的卷积神经网络作为一个任意的特征提取器。在上图中，我们可以看到，左边的是经典的VGG16网络架构，输出的是1000个ImageNet中的类别（class）的概率。右边的是去掉全连接层的VGG16网络，输出的是最后一个池化层。这时，我们就可以将输出作为我们提取的特征。

事实上，不仅是在最后的全连接层，我们可以在任意的一层停止正向传播，从而获取那一层输出的值作为我们提取的特征矢量（feature vector）。如果只去掉最后的全连接层，最后一层将变成最大池化层，输出形状为7\*7\*512的特征矢量。当我们或者这些特征矢量之后，我们就可以将他们输入进支持向量机，逻辑回归等机器学习模型中，从而获得一个新的分类器来区分目标物体类别。

### 微调（fine-tuning）

微调主要是将预训练模型顶端原有的一个或多个全连接层去掉，替换成新的一个或多个全连接层并对这些权重进行*微调*，同时在原有模型和全连接层之间也可以加入新的卷积层等常用层。

当使用特征提取的时候，我们并没有重新训练卷积神经网络，而是把它简单的当做一个特征提取器。而微调不仅需要我们更新网络的结构，还需要我们重新训练它去学习新的类别。值得注意的是，在微调时时非常容易产生过拟合的情况。

微调一般包含以下步骤：
* 移除全连接层并初始化新的全连接层
* 冻结网络中之前的卷积层，从而保护之前在预训练中习得的特征不被损坏
* 仅训练全连接层
* 可依据情况选择解冻部分卷积层进行第二次训练

在初始化新的全连接层后训练网络时，一般使用较低的学习率（learning rate），从而使新的全连接层能学习到之前的卷积层中的图案（pattern），这个过程叫做给全连接层”热身“。通过微调的迁移学习，我们通常会获得比通过特征提取的迁移学习更高的准确率。

在下图中，我们可以看到，左边的是VGG16的网络架构。中间是之前提到的，通过去掉全连接层来使用最后池化层的输出作为特征提取器。而右边的是微调的操作，通过初始化新的全连接层来替换掉之前的全连接层，从而在训练之后可以输出与训练集相关的类别。

<div>
<img src="https://pyimagesearch.com/wp-content/uploads/2019/06/fine_tuning_keras_network_surgery.png" width="600"/>
</div>

下面的这张图中展示了冻结网络中之前的卷积层。左边的网络架构显示，在我们开始微调过程时，我们先将所有的卷积层冻结，只允许梯度通过新的全连接层反向传播。右边的网络架构显示，在全连接层”热身“结束之后，我们可以解冻全部（或部分）卷积层来对每一（部分）层进行微调。

<div>
<img src="https://pyimagesearch.com/wp-content/uploads/2019/06/fine_tuning_keras_freeze_unfreeze.png" width="600"/>
</div>

在某些案例中，我们不需要解冻之前的卷积层就可以获得足够的准确率。但在大多数情况下，我们都需要解冻卷积层来获得更高的准确率。在解冻卷积层之后的训练中，我们需要使用非常小的学习率来保证不大幅度的改变卷积层。

### 代码

以下代码的背景是，使用在ImageNet上预训练的VGG16模型，来识别11种不同种类的食物。详见[原博客](https://www.pyimagesearch.com/2019/06/03/fine-tuning-with-keras-and-deep-learning/)

```python
# USAGE
# python train.py

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from pyimagesearch import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

# 一个可以绘制和保存训练历史图像的帮助方程
def plot_training(H, N, plotPath):
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
	plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
	plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
	plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

# 从config里导出训练/验证/测试集的路径
trainPath = os.path.sep.join([config.BASE_PATH, config.TRAIN])
valPath = os.path.sep.join([config.BASE_PATH, config.VAL])
testPath = os.path.sep.join([config.BASE_PATH, config.TEST])

# 计算训练/验证/测试集中图片的数量
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

# 初始化一个针对训练集的图像增强对象
# 图像增强在大多数情况下都是建议采用的策略，通过旋转，放大，扭曲，翻转等手段，可以使训练出来的模型具有更高的鲁棒性
# 值得注意的是， 图像增强不会增加数据量，将增强过的图片加入到原数据集是错误的做法
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# 初始化一个针对验证集的图像增强对象
# 因为我们不准备对验证集进行图像增强，所以这里没有输入任何变量，这个图像增强对象主要是为了方便以后进行均值消减（mean subtraction）
valAug = ImageDataGenerator()

# 定义ImageNet的均值消减（按照RGB的顺讯）并将均值输入到训练和验证集的图像增强对象中
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

# 初始化训练/验证/测试数据生成器，生成器可以按照对应路径加载每批次（batch)的图片
# 这样做的好处是不会因为同时加载所有的数据而导致RAM不够用
trainGen = trainAug.flow_from_directory(
	trainPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=config.BATCH_SIZE)

valGen = valAug.flow_from_directory(
	valPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BATCH_SIZE)

testGen = valAug.flow_from_directory(
	testPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=config.BATCH_SIZE)

########################### 微调从这里开始 ###########################

# 加载VGG16网络（带有ImageNet预训练的weights），加载的时候去掉网络的头，也就是全连接层
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# 创建作为替代的全新的全连接层
headModel = baseModel.output  # 从剪裁过得原网络中获得输出作为输入
headModel = Flatten(name="flatten")(headModel)  # 将输入扁平化
headModel = Dense(512, activation="relu")(headModel)  # 加入一个紧密相连（dense）的神经网络层，512个神经元，用relu激活
headModel = Dropout(0.5)(headModel)  # 随机抛弃（dropout）一半的输入，这一步可以帮助防止过拟合
headModel = Dense(len(config.CLASSES), activation="softmax")(headModel)  # 再加入一个神经网络层，神经元数量跟类别（class）
                                                                         # 数量一致，用softmax激活

# place the head FC model on top of the base model (this will become
# the actual model we will train)
# 把新的全连接层与剪裁过的模型连接（通过定义outputs）
model = Model(inputs=baseModel.input, outputs=headModel)

# 将VGG16模型中所有卷积层冻结（不能在训练时被更新）
for layer in baseModel.layers:
	layer.trainable = False

# 编译模型：优化器选择随机梯度下降（Stochastic Gradient Descent）, 使用categorical_crossentropy是因为输出有多个标签
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# 训练模型时用到了之前定义的数据增强对象，每一个epoch的步数也使用了config里相对应的值
# 这里先用50个epoch，单独训练全连接层。目的是为了让全连接层不是完全随机，而是能学到一些训练集中的特征。
# 学习的程度肯定不足以完成微调，所谓的学习更像是在50个epoch之后，使用了训练集中的特征来初始化全连接层。
print("[INFO] training head...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // config.BATCH_SIZE,
	validation_data=valGen,
	validation_steps=totalVal // config.BATCH_SIZE,
	epochs=50)

# 评估仅对于全连接层训练50个epoch之后的微调结果
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // config.BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
plot_training(H, 50, config.WARMUP_PLOT_PATH)

# 输出结果
#                precision    recall  f1-score   support
#         Bread       0.79      0.52      0.62       368
# Dairy product       0.75      0.55      0.64       148
#       Dessert       0.71      0.68      0.69       500
#           Egg       0.68      0.78      0.72       335
#    Fried food       0.64      0.74      0.68       287
#          Meat       0.73      0.88      0.79       432
#       Noodles       0.94      0.95      0.95       147
#          Rice       0.92      0.89      0.90        96
#       Seafood       0.80      0.82      0.81       303
#          Soup       0.92      0.94      0.93       500
#     Vegetable       0.89      0.84      0.86       231
#
#     micro avg       0.78      0.78      0.78      3347
#     macro avg       0.80      0.78      0.78      3347
#  weighted avg       0.78      0.78      0.77      3347
```

在只训练了全连接层之后，准确率大概在78%左右
<div>
<img src="https://pyimagesearch.com/wp-content/uploads/2019/06/warmup.png" width="500"/>
</div>

```python

# 重置训练/验证集的数据生成器
trainGen.reset()
valGen.reset()

# 在训练完全连接层之后，现在解冻网络中的最后一个卷积层
for layer in baseModel.layers[15:]:
	layer.trainable = True

# 打印每一层可以训练与否的状态
for layer in baseModel.layers:
	print("{}: {}".format(layer, layer.trainable))

# 由于我们解冻了最后一个卷积层，这里需要重新编译模型，并且使用很低的learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# 在最后一个卷积层和全连接层上重新训练
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // config.BATCH_SIZE,
	validation_data=valGen,
	validation_steps=totalVal // config.BATCH_SIZE,
	epochs=20)

# 评估对于最后一个卷积层和全连接层训练20个epoch之后的微调结果
print("[INFO] evaluating after fine-tuning network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // config.BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
plot_training(H, 20, config.UNFROZEN_PLOT_PATH)

# 输出结果
#                precision    recall  f1-score   support
#         Bread       0.86      0.78      0.82       368
# Dairy product       0.85      0.65      0.74       148
#       Dessert       0.83      0.79      0.81       500
#           Egg       0.84      0.84      0.84       335
#    Fried food       0.75      0.92      0.82       287
#          Meat       0.89      0.88      0.88       432
#       Noodles       0.99      0.95      0.97       147
#          Rice       0.88      0.95      0.91        96
#       Seafood       0.86      0.91      0.88       303
#          Soup       0.97      0.95      0.96       500
#     Vegetable       0.86      0.96      0.91       231
#
#     micro avg       0.87      0.87      0.87      3347
#     macro avg       0.87      0.87      0.87      3347
#  weighted avg       0.87      0.87      0.87      3347
```
为了不过拟合，作者只训练了20个epoch。如下图所示，当training loss快速下降，validation loss停滞甚至上升时，就意味着过拟合。总的来说，在将最后一个卷积层带入训练时，准确率提高到了87%，对于只训练全连接层是一个进步。
<div>
<img src="https://pyimagesearch.com/wp-content/uploads/2019/06/unfrozen.png" width="500"/>
</div>

```python
# 保存模型
print("[INFO] serializing network...")
model.save(config.MODEL_PATH)
```

### 思考

博客中这个案例并不能很好的体现我们的情况，但也提供了一些思路。
1. 我们在fine-tuning的时候，可以借鉴一些作者如何调用Keras中的模型和层（如将VGG16换成MobileNet），从而快速的修改已有模型来达到训练目的。SSD也有Keras Implementation（如[ssd_keras](https://github.com/pierluigiferrari/ssd_keras)）
2. 在我们实际操作时，要搞清楚解冻更多的层，甚至不冻结任何层对过拟合的情况到底是有帮助还是会更加恶化？有没有必要初始化新的全连接层?

这里是MobileNetV3的一部分代码，我们看到了同样在文中出现的include_top。结合起来看，我们就可以了解到在MobileNetV3中网络头部的结构。
```python
if include_top:
    x = layers.GlobalAveragePooling2D()(x)
    if channel_axis == 1:
        x = layers.Reshape((last_conv_ch, 1, 1))(x)
    else:
        x = layers.Reshape((1, 1, last_conv_ch))(x)
    x = layers.Conv2D(last_point_ch,
                      kernel_size=1,
                      padding='same',
                      name='Conv_2')(x)
    x = layers.Activation(activation)(x)
    if dropout_rate > 0:
        x = layers.Dropout(dropout_rate)(x)
    x = layers.Conv2D(classes,
                      kernel_size=1,
                      padding='same',
                      name='Logits')(x)
    x = layers.Flatten()(x)
    x = layers.Softmax(name='Predictions/Softmax')(x)
else:
    if pooling == 'avg':
        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    elif pooling == 'max':
        x = layers.GlobalMaxPooling2D(name='max_pool')(x)
```